# <font color="Blue">Abstract [EDIT NEEDED]</font>

In this notebook, we developed a prompt template to generate different types of interview questions: direct questions, behavioral questions, and technical questions.

**<font color="red">Our goal</font>** is to craft an effective prompt tailored to providing different types of interview questions in the context of specific jobs and companies.

<font color="green"> Conclusion: </font>  


# Prepare the notebook for use

## Install packages & import libraries

In [ ]:
# Download required packages to Colab
!pip install -q langchain
!pip install -q openai
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    ret

In [ ]:
# Import required libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain import ConversationChain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import os
from getpass import getpass

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

import pandas as pd
import csv

import re

ModuleNotFoundError: ignored

## OpenAI API key

In [ ]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


## Connect Google Drive

## Create .csv files to collect info from this notebook

Since the results we got from LLMs will be different everytime, we want to use .csv files to track those results which could help use go back to see what we can do or what we did to improve the outputs.

In [ ]:
with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/diff_types_questions.csv', 'w', newline='') as csvfile:
    fieldnames = ['comb_num','var_comb_num','types','question']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

In [ ]:
with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/diff_types_questions_prompt_result.csv', 'w', newline='') as csvfile:
    fieldnames = ['model', 'variable_combination', 'question_type','template', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

In [ ]:
var_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv")
prompt_result = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/diff_types_questions_prompt_result.csv")
type_q = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/diff_types_questions.csv")

In [ ]:
var_comb

,comb_num,interviewer_role,company_name,resume,role_title,role_description,company_description
0,1,recruiter,S&P Global,\nEDUCATION\nVanderbilt University – Data Scie...,Data Engineer - AI/ML (Virtual),\nJob Description\n\nThis is a software develo...,\nOverview\nS&P Global (NYSE: SPGI) provides e...


In [ ]:
prompt_result

,model,variable_combination,question_type,template,output


In [ ]:
type_q

,comb_num,var_comb_num,types,question


## Read data

In [ ]:
file = open("/content/drive/MyDrive/Capstone Project - Mock Interview App/data/different_interview_questions_definition.txt", "r")

In [ ]:
diff_def = file.read()
print(diff_def)

Understanding the differences between direct, behavioral, and technical interview questions is crucial for effective interview preparation. Each type serves a specific purpose in evaluating different aspects of a candidate's qualifications and suitability for a role.

1. Direct Interview Questions:
Purpose: These questions are straightforward and are designed to gather factual information about the candidate's background, qualifications, and work experience.
Characteristics: Direct questions are usually clear-cut and require specific, concise answers. They don't typically require candidates to elaborate on scenarios or demonstrate problem-solving skills.

2. Behavioral Interview Questions:
Purpose: These questions are aimed at understanding how a candidate has behaved in specific situations in the past, based on the belief that past behavior is a good predictor of future behavior.
Characteristics: Behavioral questions often start with phrases like "Tell me about a time when..." or "Giv

# Model selection

In [ ]:
chat = ChatOpenAI(model_name = "gpt-4")

In [ ]:
# record model as ex_model
ex_model = "gpt_4"

# Different types interview questions prompt

In [ ]:
var_comb_num = 1

## Prompt 1

### prompt template

In [ ]:
input_var_1 = ['role_title', 'company_name',
               'question_type',
               'role_description', 'company_description']

In [ ]:
template_1 = """
As an experienced interviewer for {role_title} at {company_name}, what are the
likely {question_type} interview questions you would like to ask?

Here is the job description: {role_description}
Here is the company description: {company_description}

Here are definitions for different types of interview questions:
Understanding the differences between direct, behavioral, and technical interview questions is crucial for effective interview preparation. Each type serves a specific purpose in evaluating different aspects of a candidate's qualifications and suitability for a role.
1. Direct Interview Questions:
Purpose: These questions are straightforward and are designed to gather factual information about the candidate's background, qualifications, and work experience.
Characteristics: Direct questions are usually clear-cut and require specific, concise answers. They don't typically require candidates to elaborate on scenarios or demonstrate problem-solving skills.
2. Behavioral Interview Questions:
Purpose: These questions are aimed at understanding how a candidate has behaved in specific situations in the past, based on the belief that past behavior is a good predictor of future behavior.
Characteristics: Behavioral questions often start with phrases like "Tell me about a time when..." or "Give me an example of...". They require candidates to tell stories or anecdotes from their past experiences, demonstrating their skills, decision-making abilities, and how they handle work-related situations.
3. Technical Interview Questions:
Purpose: These questions assess the candidate's technical skills and knowledge pertinent to the specific role they are applying for. They are common in fields like IT, engineering, science, or any role that requires specific technical expertise.
Characteristics: Technical questions are often quite specific and can include problem-solving or even practical demonstrations of skills. They may involve scenarios, hypothetical problems, coding tests, or detailed discussions of technologies or methodologies.
Each type of question helps the interviewer gauge different aspects of a candidate's profile. Direct questions assess basic qualifications, behavioral questions evaluate soft skills and cultural fit, and technical questions measure job-specific skills and expertise.
"""

In [ ]:
prompt_1 = PromptTemplate(
    input_variables = input_var_1,
    template = template_1
)

### parameter def

In [ ]:
job_title = var_comb.role_title[0]
comp_name = var_comb.company_name[0]

job_des = var_comb.role_description[0]
comp_des = var_comb.company_description[0]

### DIRECT

In [ ]:
q_type = "direct"

In [ ]:
new_prompt = prompt_1.format(role_title=job_title,
                             company_name=comp_name,
                             question_type=q_type,
                             role_description=job_des,
                             company_description=comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

1. Can you describe your experience with software development and testing? 
2. Have you worked with machine learning algorithms before? If so, could you describe a project where you implemented them?
3. Can you talk about your experience in collecting and cleaning data for AI/ML models?
4. How do you evaluate the performance of AI/ML models? What metrics do you typically use?
5. Can you share an example when you had to work closely with stakeholders to understand their business needs and develop a solution around those needs?
6. What is your experience with deploying AI models? Can you give a specific example?
7. Can you describe a situation where you had to work closely with a QA team to fix bugs? What was the outcome?
8. How familiar are you with automation in the context of software development?
9. What programming languages are you proficient in? Could you describe a project where you used Python or Java?
10. How do you stay up-to-date with emerging technologies in the AI/ML field?

#### store data (prompt-result)

In [ ]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [var_comb_num],
    'question_type': [q_type],
    'template': [template_1],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [ ]:
prompt_result

,model,variable_combination,question_type,template,output
0,gpt_4,1,direct,\nAs an experienced interviewer for {role_titl...,1. Can you describe your experience with softw...


#### store data (type-questions)

In [ ]:
result_string = """
1. Can you describe your experience with software development and testing?
2. Have you worked with machine learning algorithms before? If so, could you describe a project where you implemented them?
3. Can you talk about your experience in collecting and cleaning data for AI/ML models?
4. How do you evaluate the performance of AI/ML models? What metrics do you typically use?
5. Can you share an example when you had to work closely with stakeholders to understand their business needs and develop a solution around those needs?
6. What is your experience with deploying AI models? Can you give a specific example?
7. Can you describe a situation where you had to work closely with a QA team to fix bugs? What was the outcome?
8. How familiar are you with automation in the context of software development?
9. What programming languages are you proficient in? Could you describe a project where you used Python or Java?
10. How do you stay up-to-date with emerging technologies in the AI/ML field?
11. Can you explain a complex machine learning concept in simple terms?
12. Given your understanding of S&P Global, how do you think AI/ML can contribute to our business goals?
13. How do you handle data privacy and security when developing AI/ML models?
14. Are you comfortable working in a flexible, remote work environment?
15. Can you discuss your educational background and how it has prepared you for this role?
"""

result_split = result_string.split("\n")[1:16]
result_questions = [re.sub('^\d{0,2}.\ ', '', i) for i in result_split]
result_questions

['Can you describe your experience with software development and testing? ',
 'Have you worked with machine learning algorithms before? If so, could you describe a project where you implemented them?',
 'Can you talk about your experience in collecting and cleaning data for AI/ML models?',
 'How do you evaluate the performance of AI/ML models? What metrics do you typically use?',
 'Can you share an example when you had to work closely with stakeholders to understand their business needs and develop a solution around those needs?',
 'What is your experience with deploying AI models? Can you give a specific example?',
 'Can you describe a situation where you had to work closely with a QA team to fix bugs? What was the outcome?',
 'How familiar are you with automation in the context of software development?',
 'What programming languages are you proficient in? Could you describe a project where you used Python or Java?',
 'How do you stay up-to-date with emerging technologies in the AI/ML

In [ ]:
new_type_q = pd.DataFrame({
    'comb_num': list(range(1,16)),
    'var_comb_num': [var_comb_num]*15,
    'types': [q_type]*15,
    'question': result_questions
})

type_q = pd.concat([type_q, new_type_q], ignore_index=True)

In [ ]:
type_q

,comb_num,var_comb_num,types,question
0,1,1,direct,Can you describe your experience with software...
1,2,1,direct,Have you worked with machine learning algorith...
2,3,1,direct,Can you talk about your experience in collecti...
3,4,1,direct,How do you evaluate the performance of AI/ML m...
4,5,1,direct,Can you share an example when you had to work ...
5,6,1,direct,What is your experience with deploying AI mode...
6,7,1,direct,Can you describe a situation where you had to ...
7,8,1,direct,How familiar are you with automation in the co...
8,9,1,direct,What programming languages are you proficient ...
9,10,1,direct,How do you stay up-to-date with emerging techn...


### BEHAVIORAL

In [ ]:
q_type = "behavioral"

In [ ]:
new_prompt = prompt_1.format(role_title=job_title,
                             company_name=comp_name,
                             question_type=q_type,
                             role_description=job_des,
                             company_description=comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

1. Can you tell me about a time when you had to use your problem-solving skills to fix a bug in an AI/ML model? How did you identify the problem and what was your process to fix it?

2. Describe a situation where you had to collect and clean data for a machine learning project. What were the challenges and how did you overcome them?

3. Can you share an example of a project where you had to work closely with stakeholders to understand their business needs and develop an AI/ML solution accordingly?

4. Give me an example of a time you had to evaluate the performance of an AI/ML model. What metrics did you use and what were the results?

5. Can you tell me about a time when you had to work independently and as part of a team on a project? How did you manage your responsibilities?

6. Describe a time when you had to implement machine learning algorithms for a project. What was the project about and what was your role?

7. Can you share an example of a time when you had to work under tight

#### store data (prompt-result)

In [ ]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [var_comb_num],
    'question_type': [q_type],
    'template': [template_1],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [ ]:
prompt_result

,model,variable_combination,question_type,template,output
0,gpt_4,1,direct,\nAs an experienced interviewer for {role_titl...,1. Can you describe your experience with softw...
1,gpt_4,1,behavioral,\nAs an experienced interviewer for {role_titl...,1. Can you tell me about a time when you had t...


#### store data (type-questions)

In [ ]:
result_string = """
1. Can you tell me about a time when you had to use your problem-solving skills to fix a bug in an AI/ML model? How did you identify the problem and what was your process to fix it?

2. Describe a situation where you had to collect and clean data for a machine learning project. What were the challenges and how did you overcome them?

3. Can you share an example of a project where you had to work closely with stakeholders to understand their business needs and develop an AI/ML solution accordingly?

4. Give me an example of a time you had to evaluate the performance of an AI/ML model. What metrics did you use and what were the results?

5. Can you tell me about a time when you had to work independently and as part of a team on a project? How did you manage your responsibilities?

6. Describe a time when you had to implement machine learning algorithms for a project. What was the project about and what was your role?

7. Can you share an example of a time when you had to work under tight deadlines? How did you ensure that the quality of your work was not compromised?

8. Tell me about a time when you had to learn a new technology or tool for a project. How did you go about it and how did it benefit the project?

9. Share an experience where you had to work with a QA team to fix bugs. What was your approach and how did it improve the final product?

10. Describe a situation where your basic understanding of automation helped in improving a process or a project.

11. Can you tell me about a time when your passion for emerging technologies had a positive impact on your work or a project?

12. Describe a situation where you took a decision that significantly benefited your project or team. What was the situation and what was the result of your decision?

13. Can you share an example of a time when you had to work flexibly, perhaps outside of normal working hours or in a different location, to meet a project deadline or achieve a goal?

14. How have you managed to keep your technical skills up to date in the rapidly changing field of AI/ML? Can you provide examples of any recent training or learning you have undertaken?
"""

result_split = result_string.split("\n")
result_questions = [re.sub('^\d{0,2}.\ ', '', i) for i in result_split if len(i) != 0]
result_questions

['Can you tell me about a time when you had to use your problem-solving skills to fix a bug in an AI/ML model? How did you identify the problem and what was your process to fix it?',
 'Describe a situation where you had to collect and clean data for a machine learning project. What were the challenges and how did you overcome them?',
 'Can you share an example of a project where you had to work closely with stakeholders to understand their business needs and develop an AI/ML solution accordingly?',
 'Give me an example of a time you had to evaluate the performance of an AI/ML model. What metrics did you use and what were the results?',
 'Can you tell me about a time when you had to work independently and as part of a team on a project? How did you manage your responsibilities?',
 'Describe a time when you had to implement machine learning algorithms for a project. What was the project about and what was your role?',
 'Can you share an example of a time when you had to work under tight 

In [ ]:
new_type_q = pd.DataFrame({
    'comb_num': list(range(len(type_q)+1, len(type_q) + len(result_questions)+1)),
    'var_comb_num': [var_comb_num]*len(result_questions),
    'types': [q_type]*len(result_questions),
    'question': result_questions
})
#new_type_q
type_q = pd.concat([type_q, new_type_q], ignore_index=True)

In [ ]:
type_q

,comb_num,var_comb_num,types,question
0,1,1,direct,Can you describe your experience with software...
1,2,1,direct,Have you worked with machine learning algorith...
2,3,1,direct,Can you talk about your experience in collecti...
3,4,1,direct,How do you evaluate the performance of AI/ML m...
4,5,1,direct,Can you share an example when you had to work ...
5,6,1,direct,What is your experience with deploying AI mode...
6,7,1,direct,Can you describe a situation where you had to ...
7,8,1,direct,How familiar are you with automation in the co...
8,9,1,direct,What programming languages are you proficient ...
9,10,1,direct,How do you stay up-to-date with emerging techn...


### TECHNICAL

In [ ]:
q_type = "technical"

In [ ]:
new_prompt = prompt_1.format(role_title=job_title,
                             company_name=comp_name,
                             question_type=q_type,
                             role_description=job_des,
                             company_description=comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

1. In your previous experience, what kind of data did you handle and how did you ensure the data was clean and ready for use in AI/ML models?
2. Can you explain a machine learning project you have implemented from scratch? What was the problem, the approach, and the outcome?
3. How would you handle missing or corrupted data in a dataset?
4. Can you describe a situation where you had to develop a solution around the needs of a business stakeholder? What was the outcome?
5. Can you explain your approach to evaluating the performance of an AI/ML model?
6. What kind of experience do you have with cloud solutions and digital platforms? Can you provide specific examples?
7. Can you talk about your experience in software development and testing? What programming languages are you proficient in?
8. How would you handle a situation where the AI model you've developed is not delivering the expected results?
9. Can you discuss a scenario where you had to work closely with a QA team to fix bugs in

#### store data (prompt-result)

In [ ]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [var_comb_num],
    'question_type': [q_type],
    'template': [template_1],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [ ]:
prompt_result

,model,variable_combination,question_type,template,output
0,gpt_4,1,direct,\nAs an experienced interviewer for {role_titl...,1. Can you describe your experience with softw...
1,gpt_4,1,behavioral,\nAs an experienced interviewer for {role_titl...,1. Can you tell me about a time when you had t...
2,gpt_4,1,technical,\nAs an experienced interviewer for {role_titl...,"1. In your previous experience, what kind of d..."


#### store data (type-questions)

In [ ]:
result_string = result.content

result_split = result_string.split("\n")
result_questions = [re.sub('^\d{0,2}.\ ', '', i) for i in result_split if len(i) != 0]
result_questions

['In your previous experience, what kind of data did you handle and how did you ensure the data was clean and ready for use in AI/ML models?',
 'Can you explain a machine learning project you have implemented from scratch? What was the problem, the approach, and the outcome?',
 'How would you handle missing or corrupted data in a dataset?',
 'Can you describe a situation where you had to develop a solution around the needs of a business stakeholder? What was the outcome?',
 'Can you explain your approach to evaluating the performance of an AI/ML model?',
 'What kind of experience do you have with cloud solutions and digital platforms? Can you provide specific examples?',
 'Can you talk about your experience in software development and testing? What programming languages are you proficient in?',
 "How would you handle a situation where the AI model you've developed is not delivering the expected results?",
 'Can you discuss a scenario where you had to work closely with a QA team to fix 

In [ ]:
new_type_q = pd.DataFrame({
    'comb_num': list(range(len(type_q)+1, len(type_q) + len(result_questions)+1)),
    'var_comb_num': [var_comb_num]*len(result_questions),
    'types': [q_type]*len(result_questions),
    'question': result_questions
})
#new_type_q
type_q = pd.concat([type_q, new_type_q], ignore_index=True)

In [ ]:
type_q

,comb_num,var_comb_num,types,question
0,1,1,direct,Can you describe your experience with software...
1,2,1,direct,Have you worked with machine learning algorith...
2,3,1,direct,Can you talk about your experience in collecti...
3,4,1,direct,How do you evaluate the performance of AI/ML m...
4,5,1,direct,Can you share an example when you had to work ...
5,6,1,direct,What is your experience with deploying AI mode...
6,7,1,direct,Can you describe a situation where you had to ...
7,8,1,direct,How familiar are you with automation in the co...
8,9,1,direct,What programming languages are you proficient ...
9,10,1,direct,How do you stay up-to-date with emerging techn...


In [ ]:
type_q

NameError: ignored